### Importing important Libraries

In [1]:
import pandas as pd
import pymysql as sql
import numpy as np

### Connecting to the cloud and checking connection..

In [2]:
## Connecting to the AWS Database

host_name = "#############"
dbname = '#############'
port = 3306
username = '#############'
password = '#############'

conn = sql.connect(user=username, password=password, host=host_name,database=dbname,port=port)
curr = conn.cursor()

In [3]:
curr.execute("""SELECT * FROM videos;""")
curr.fetchall()

(('6ilZeMFyNE8',
  'LN+ EN VIVO las 24 horas',
  '0000-00-00',
  9714535,
  25110,
  3463,
  0),
 ('F2yswKCPGoI',
  'Jimmy Kimmel Has Had Enough of Scumbags Attacking Dr. Fauci for Trying to Keep Us Alive',
  '0000-00-00',
  1197109,
  35000,
  2309,
  4126))

### Importing the file that is to be uploaded to the cloud database...

In [4]:
df=pd.read_csv("/Users/1kaptaan/Downloads/ML_Practice/Ecommerce_Customers.csv")
df.head(3)

,Email,Address,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank TunnelWrightmouth, MI 82180-9605",34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer CommonDiazchester, CA 06566-8576",31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582Cobbborough, DC ...",33.000915,11.330278,37.110597,4.104543,487.547505


### Create a table in the cloud database to store all the values from our file...

In [5]:
curr.execute("""DROP TABLE ecom_customer""")

0

In [6]:
# Create a table in the AWS database to store all the data...

curr.execute("""CREATE TABLE IF NOT EXISTS ecom_customer(
                Email TEXT NOT NULL,
                Address TEXT NOT NULL,
                Avg_Session_Length FLOAT NOT NULL,
                Time_on_App FLOAT NOT NULL,
                Time_on_Website FLOAT NOT NULL,
                Length_of_Membership FLOAT NOT NULL,
                Yearly_Amount_Spent FLOAT NOT NULL
                )""")

0

In [7]:
curr.execute("""SELECT * FROM ecom_customer""")
curr.fetchall()

()

### Create a function to automate changing column name and dtypes..

In [8]:
# Changing column name and dtypes with one function...
def change_col_dtypes(df):
    replacement = {'Email':'string','Address':'string','Avg_Session_Length':'float','Time_on_App':'float',
                   'Time_on_Website':'float','Length_of_Membership':'float','Yearly_Amount_Spent':'float'}
    
    col_replacement = {'Avg Session Length':'Avg_Session_Length','Time on App':'Time_on_App',
                   'Time on Website':'Time_on_Website','Length of Membership':'Length_of_Membership',
                  'Yearly Amount Spent':'Yearly_Amount_Spent'}
    for col in df:
        df = df.rename(columns= col_replacement)
        df = df.astype(replacement)
    return df

In [9]:
df = change_col_dtypes(df)

In [10]:
df.dtypes

Email                    string
Address                  string
Avg_Session_Length      float64
Time_on_App             float64
Time_on_Website         float64
Length_of_Membership    float64
Yearly_Amount_Spent     float64
dtype: object

### We create different loops to check, update and insert a new entry to the cloud...

In [11]:
def check_if_exist(curr,Email):
    query = ("""SELECT Email FROM ecom_customer WHERE Email = %s""")
    curr.execute(query,(Email))
    return curr.fetchone() is not None

In [12]:
def update(curr, Email, Address, Avg_Session_Length, Time_on_App, Time_on_Website, 
           Length_of_Membership, Yearly_Amount_Spent):
    query = ("""UPDATE ecom_customer SET
            Address = %s,
            Avg_Session_Length = %s,
            Time_on_App = %s,
            Time_on_Website = %s,
            Length_of_Membership = %s,
            Yearly_Amount_Spent = %s
            WHERE Email = %s""")
    var_update=(Address, Avg_Session_Length, Time_on_App, Time_on_Website, Length_of_Membership,
                Yearly_Amount_Spent, Email)
    curr.execute(query,var_update)

In [13]:
def df_update(curr,df):
    update_df = pd.DataFrame(columns=['Email','Address','Avg_Session_Length','Time_on_App','Time_on_Website',
                                  'Length_of_Membership','Yearly_Amount_Spent'])

    for i, row in df.iterrows():
        if check_if_exist(curr,row['Email']):
            update(curr,row['Email'],row['Address'],row['Avg_Session_Length'],row['Time_on_App'],
                   row['Time_on_Website'],row['Length_of_Membership'],row['Yearly_Amount_Spent'])
        else:
            update_df = update_df.append(row)
    return update_df        

In [14]:
updated_data = df_update(curr,df)

### After extracting the update df we change the data types so that they match the table in cloud...

### Now we define insert function to update entries in cloud database...

In [15]:
def insert_db(curr,Email,Address,Avg_Session_Length,Time_on_App,Time_on_Website,
                 Length_of_Membership,Yearly_Amount_Spent):
    query = ("""INSERT INTO ecom_customer
                (Email, Address, Avg_Session_Length, Time_on_App, Time_on_Website,
                Length_of_Membership, Yearly_Amount_Spent)
                VALUES(%s,%s,%s,%s,%s,%s,%s)""")
    rows_update = (Email,Address,Avg_Session_Length,Time_on_App,Time_on_Website,
                     Length_of_Membership,Yearly_Amount_Spent)
    curr.execute(query,rows_update)

In [16]:
def df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_db(curr,row['Email'],row['Address'],row['Avg_Session_Length'],row['Time_on_App'],
                  row['Time_on_Website'],row['Length_of_Membership'],row['Yearly_Amount_Spent'])

In [17]:
df_to_db(curr, updated_data)

In [18]:
conn.commit()